# LLM Workflow Design Patterns: Prompt Chaining
- This file is a project for applying design pattern: prompt-chaining
- How to use:
1. Fill the article.txt file with the article/text you would like to generate the quiz from
2. Choose your LLM model (mine is gemini 2.5 flash)
3. Change number of questions inside the main function

``` python
def main():
    with open("./article.txt", "r", encoding="utf-8") as f:
        article = f.read()
    summary = summarize(article)
    keywords = extract_keywords(summary)
    quiz = generate_quiz(keywords=keywords, num_questions=5) # change here
    display_quiz(quiz)
```

<br>

![Prompt Chaining Diagram](../../assets/prompt_chaining.png)

## Project Idea: Article Summarizer and Quiz Generator

##### Workflow (Prompt Chain)
1. Step 1: Summarize
2. Step 2: Extract Key Terms
3. Step 3: Generate Quiz Questions

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv(override=True)

GEMINI_KEY = os.getenv("GOOGLE_API_KEY")

if GEMINI_KEY:
    print("Gemini key loaded successfully")
else:
    print("Failed to load Gemini key.")
    exit(-1)

GEMINI_URL = os.getenv("GOOGLE_API")

if GEMINI_URL:
    print("Gemini URL loaded successfully")
else:
    print("Failed to load Gemini URL.")
    exit(-1)

client =  OpenAI(base_url=GEMINI_URL, api_key=GEMINI_KEY)

Gemini key loaded successfully
Gemini URL loaded successfully


### Step 1: Summarize the text

In [3]:
def summarize(text: str) -> str:
    content = "I will give you a text that you need to summarize, make sure to keep keywords and coherence of the text\n"
    content += f"The text: {text}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    summary = response.choices[0].message.content 
    
    return summary

### Step 2: Extract keywords

In [4]:
import json

def extract_keywords(summary: str) -> list:
    content = f'I will give you a text summary, i need you to extract keywords regarding the topic for a quiz. Be precise and topic-oriented, and ignore general terms. Do not include any markdown. Respond with JSON, and only JSON, with the following format: {{"keywords": [keyword1, keyword2, ...]}}\n'
    content += f"The summary: {summary}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)
    result = response.choices[0].message.content

    keywords_dict = json.loads(result)

    return keywords_dict["keywords"]


### Step 3: Generate quiz

In [5]:
def generate_quiz(keywords: list, num_questions: int = 10) -> list:
    content = f'I will give you a list of keywords, I want you to generate a {num_questions} question quiz about all keywords ({num_questions} questions total). Do not include any markdown or extra text or text styling. Return the quiz in a list of JSONs format [{{"question": ..., "answer": ...}}]\n'
    content += f"The keywords list: {keywords}"

    messages = [{"role": "user", "content": content}]

    response = client.chat.completions.create(model="gemini-2.5-flash", messages=messages)

    quiz_list = json.loads(response.choices[0].message.content)

    return quiz_list

### Display the quiz

In [6]:
def display_quiz(quiz: list):
    for index, question in enumerate(quiz):
        print(f"Q{index + 1}: {question['question']}")
        print(f"A: {question['answer']}\n")

### Main Program

In [7]:
def main():
    with open("./article.txt", "r", encoding="utf-8") as f:
        article = f.read()

    summary = summarize(article)

    keywords = extract_keywords(summary)

    quiz = generate_quiz(keywords=keywords, num_questions=5)

    display_quiz(quiz)
    

In [8]:
main()

Q1: What are LLMs, and how do 'tokens' and the 'limited context window' influence the 'inference' process?
A: LLMs are large language models that process text in units called tokens. The limited context window restricts the total number of tokens (input + output) an LLM can consider at once, directly impacting the scope and coherence of the inference (the generation of output) by determining how much past information is available.

Q2: Explain the relationship between 'prompt engineering,' 'context engineering,' and 'system instructions' in guiding an LLM's behavior.
A: Prompt engineering involves crafting effective prompts for an LLM. Context engineering is a broader strategy that includes prompt engineering, but also encompasses providing relevant 'system instructions' (setting the model's persona, rules, or goals) and managing other input like 'message history' to establish the overall context and guide the LLM's behavior more effectively.

Q3: How do 'tools' and 'external data' enh